In [3]:
import tensorflow as tf

In [4]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "C:\shyam\A multiclass disease classifier\data\kidney\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
    shuffle=True,
    batch_size=32,
    image_size=(299, 299),
)

labels = dataset.class_names
labels

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Shyam Kadiwar\AppData\Local\Temp\ipykernel_10212\3274122944.py:2: SyntaxWarning: invalid escape sequence '\s'
  "C:\shyam\A multiclass disease classifier\data\kidney\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",


Found 12446 files belonging to 4 classes.


['Cyst', 'Normal', 'Stone', 'Tumor']

In [5]:
import numpy as np

for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())
    break

(32, 299, 299, 3)
[1 1 0 3 1 0 3 3 1 1 1 3 1 3 0 0 0 1 3 1 1 1 1 0 0 1 1 0 0 1 1 1]


In [6]:
# train test split
train_size = int(0.8 * len(dataset))
test_size = int(0.2 * len(dataset))
train_size, test_size

(311, 77)

In [7]:
def get_dataset_partisions_tf(ds, train_split=0.8, test_split=0.2, shuffle=True, shuffle_size=10000):
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    train_size = int(train_split * len(ds))
    test_size = int(test_split * len(ds))
    train_ds = ds.take(train_size)
    test_ds = ds.skip(train_size)
    val_ds = test_ds.skip(test_size)
    test_ds = test_ds.take(test_size)
    return train_ds, test_ds, val_ds

In [8]:
train_ds, test_ds, val_ds = get_dataset_partisions_tf(dataset)
len(train_ds), len(test_ds), len(val_ds)

(311, 77, 1)

In [9]:
#Resize and Normalize
#Xception models takes the image input as 299x299 pixels so converting into the trainable format is necessary
#The Images are to be normalized before to train accurately and efficiently


In [10]:
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.Resizing(299, 299),
    tf.keras.layers.Rescaling(1./255)
])

In [11]:
# train using Xception
base_model = tf.keras.applications.Xception(
    weights='imagenet',
    input_shape=(299, 299, 3),
    include_top=False,
    pooling='avg',
    classifier_activation='softmax',
    classes=len(labels)
)

In [12]:
base_model.trainable = False

inputs = tf.keras.Input(shape=(299, 299, 3))
x = resize_and_rescale(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(len(labels), activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    batch_size=32,
    epochs=5
)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 299, 299, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 299, 299, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception (Functional)           │ (None, 2048)           │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,124,268 (80.58 MB)

 Trainable params: 262,788 (1.00 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

Epoch 1/5
311/311 ━━━━━━━━━━━━━━━━━━━━ 2785s 9s/step - accuracy: 0.7324 - loss: 0.6938 - val_accuracy: 0.9688 - val_loss: 0.2026
Epoch 2/5
311/311 ━━━━━━━━━━━━━━━━━━━━ 2597s 8s/step - accuracy: 0.9426 - loss: 0.1938 - val_accuracy: 1.0000 - val_loss: 0.0690
Epoch 3/5
311/311 ━━━━━━━━━━━━━━━━━━━━ 2465s 8s/step - accuracy: 0.9751 - loss: 0.0860 - val_accuracy: 0.9688 - val_loss: 0.1018
Epoch 4/5
311/311 ━━━━━━━━━━━━━━━━━━━━ 2460s 8s/step - accuracy: 0.9869 - loss: 0.0501 - val_accuracy: 1.0000 - val_loss: 0.0172
Epoch 5/5
311/311 ━━━━━━━━━━━━━━━━━━━━ 2508s 8s/step - accuracy: 0.9926 - loss: 0.0345 - val_accuracy: 1.0000 - val_loss: 0.0465


In [17]:
model.save('kidney_stone_detection_model.h5')